Hello,

This tutorial is focusing on understanding connection between grpc and sql(we are going to use sqlite for this tutorial).
grpc can be implemented in various languages. we are using python for this tutorial.
Details of grpc can be found at https://grpc.io

Let's begin with installation of grpc packages. 

In [1]:
!pip install grpcio

In [2]:
!pip install grpcio-tools

After successful installation of the grpc packages, we will start creating the protocol buffer file which contains declaration of service interface and the message which will be used for the client server communication.

Create proto file titled employee.proto with the below mentioned content

In [ ]:
# syntax = "proto3";

package employee;

// Interface exported by the server.
service Employee {
    rpc Read(readRequestPB) returns (readResponseListPB) {}
}

message readRequestPB {
    string first_name = 1;
    string last_name = 2;
    string email = 3;
    string gender = 4;
    string ip_address = 5;
    string country = 6;
    string postcode = 7;
}

message readResponsePB {
    string first_name = 1;
    string last_name = 2;
    string email = 3;
    string gender = 4;
    string ip_address = 5;
    string country = 6;
    string postcode = 7;
    int32 id = 8;
}

message readResponseListPB {
    repeated readResponsePB employee = 1;
}

Compile proto file and generate stub for python. Here is the command to generate:
>python -m grpc_tools.protoc -I ./protos/. --python_out=./generated/. --grpc_python_out=./generated/. ./protos/employee.proto

This will generate two files(employee_pb2.py and employee_pb2_grpc.py) under generated folder. 

Now we will start implementing server. create a file under src folder named server.py with the following content:

In [ ]:
"""
#TODO: This cell can be further broken down with details about the 
import, class and serve construction.
"""

from __future__ import print_function

import time
import grpc

from generated import employee_pb2_grpc
from concurrent import futures

_ONE_DAY_IN_SECONDS = 60 * 60 * 24

class EmployeeServicer(employee_pb2_grpc.EmployeeServicer):
    def __init__(self):
        pass
    
def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    employee_pb2_grpc.add_EmployeeServicer_to_server(EmployeeServicer(), server)
    server.add_insecure_port('[::]:40084')
    server.start()
    print("Employee server running on 40084...")
    try:
        while True:
            time.sleep(_ONE_DAY_IN_SECONDS)
    except:
        server.stop(0)

if __name__ == '__main__':
    serve()

Client code is as mentioned below in client.py

In [ ]:
from __future__ import print_function

import time
import grpc

from generated import employee_pb2, employee_pb2_grpc

class EmployeeClient(object):
    def __init__(self):
        pass

def run():
    channel = grpc.insecure_channel('localhost:40084')
    stub = employee_pb2_grpc.EmployeeStub(channel)

if __name__ == '__main__':
    run()

So far we have created server and client file with the basic connection. However, client is not calling anything 
from server yet. Let's implement that...

In [ ]:
class EmployeeClient(object):
    def __init__(self):
        pass

    def get_employee(self, stub):
        read_request = employee_pb2.readRequestPB()
        read_request.first_name = "Pauli"
        return stub.Read(read_request)
    
def run():
    channel = grpc.insecure_channel('localhost:40084')
    stub = employee_pb2_grpc.EmployeeStub(channel)

    emp_client = EmployeeClient()
    print(emp_client.get_employee(stub))

In [ ]:
class EmployeeServicer(employee_pb2_grpc.EmployeeServicer):
    def Read(self, request, context):
        print("Read employee service called...")
        response_list = employee_pb2.readResponseListPB()
        return response_list

After implementing above methods in client and server, server will return an empty list as response.